# Indexing examples for wdl file multi-dimensional Arrays
```bash
docker run -p 8888:8888 -v `pwd`:/home/tmpusr/run_dir -it aka1spanky/ncsa:devwdlV01 jupyter notebook --port=8888 --ip=0.0.0.0 --allow-root
```

In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [5]:
# running jupyter inside the container - this is the externally mounted directory
os.getcwd()

'/home/tmpusr/run_dir'

In [3]:
#           location of the cromwell files
os.listdir('/usr/locl/etc')

['womtool-36.jar', 'cromwell-36.jar', '.DS_Store']

In [9]:
#           before and after cell >> cromwell-* will show up after running the next cell
#           Iff and Only If you run the writefile below 
os.listdir()

['HelloIdxArray.wdl',
 '.ipynb_checkpoints',
 'cromwell-executions',
 'Input_Array_Indexing_Examples.ipynb',
 'cromwell-workflow-logs']

In [4]:
%%writefile HelloIdxArray.wdl
###########################
#       HelloIdxArray.wdl
# Usage: 
# java -jar full/path/to/cromwell run HelloIdxArray.wdl
###########################

task hello {
  String addressee
  Int One
  Int Two
  Int Three
  Int sIx
  command {
    echo "Input String =  ${addressee} With Iterator Index: ${sIx}  [${One}][${Two}][${Three}]"
  }
  output {
    String message = read_string(stdout())
  }
}

workflow wf_hello {

  Array[Array[Array[String]]] ImpString = [ [ ["0:0:0", "0:0:1"], ["0:1:0","0:1:1"] ],  [ ["1:0:0", "1:0:1"], ["1:1:0","1:1:1"] ] ]
  Int arr1_size = length(ImpString)
  Int arr2_size = length(ImpString[0])
  Int arr3_size = length(ImpString[0][0])

  Array[Int] Index = range(arr1_size * arr2_size * arr3_size)

  scatter (idx in Index) {

    Int reads_idx = idx % arr1_size

    Int lanes_idx = (idx / arr2_size) % arr2_size

    Int samples_idx = idx / (arr3_size * arr2_size)

    call hello as howdy {
      input:
        addressee = ImpString[samples_idx][lanes_idx][reads_idx],
        One = samples_idx,
        Two = lanes_idx,
        Three = reads_idx,
        sIx = idx
    }
  }

  output {
     howdy.message
  }
}


Writing HelloIdxArray.wdl


In [ ]:
#           construct the call string and start a new separate process
hak_str = 'java -jar /usr/locl/etc/cromwell-36.jar run HelloIdxArray.wdl'
os.system(hak_str)

In [10]:
%%writefile T2IdxArray.wdl
###########################
#              AAA_T2IdxArray.wdl
# Usage:
# java -jar full/path/to/cromwell run AAA_T2IdxArray.wdl -i AAA_json.json
# wf_hello.ImpString = [[["1"]]] 
###########################

task hello {
  String addressee
  Int One
  Int Two
  Int Three
  Int sIx
  command {
    echo "Input String =  ${addressee} With Iterator Index: ${sIx}  [${One}][${Two}][${Three}]"
  }
  output {
    String message = read_string(stdout())
  }
}

workflow wf_hello {

  Array[Array[Array[String]]] ImpString
  Int arr1_size = length(ImpString)
  Int arr2_size = length(ImpString[0])
  Int arr3_size = length(ImpString[0][0])

  Array[Int] Index = range(arr1_size * arr2_size * arr3_size)

  scatter (idx in Index) {

    Int reads_idx = idx % arr3_size

    Int lanes_idx = (idx / arr2_size) % arr2_size

    Int samples_idx = (idx / (arr3_size * arr2_size)) % arr1_size

    call hello as howdy {
      input:
        addressee = ImpString[samples_idx][lanes_idx][reads_idx],
        One = samples_idx,
        Two = lanes_idx,
        Three = reads_idx,
        sIx = idx
    }
  }

  output {
     howdy.message
  }
}


Writing T2IdxArray.wdl


In [12]:
%%writefile AAA_json.json
{
    "wf_hello.ImpString": [ 
                          [ ["0:0:0", "0:0:1"], 
                            ["0:1:0","0:1:1"] ],  
                          [ ["1:0:0", "1:0:1"], 
                            ["1:1:0","1:1:1"] ], 
                          [ ["2:0:0", "2:0:1"], 
                            ["2:1:0","2:1:1"] ] ]
}


Writing AAA_json.json


In [14]:
#           construct the call string and start a new separate process
hak_str = 'java -jar /usr/locl/etc/cromwell-36.jar run T2IdxArray.wdl -i AAA_json.json'
os.system(hak_str)

0

In [16]:
#           before and after cell >> cromwell-* will show up after running the next cell
#           Iff and Only If you run the writefile below 
os.listdir('cromwell-executions/wf_hello')

['c773d129-8092-4095-876a-10e7c70b555f',
 'cae6045a-41cb-4d28-8934-b3ba880bffe4']